In [1]:
import ipywidgets
import time
import matplotlib.pyplot as plt
import random
from mplsoccer import Radar
from IPython.display import clear_output as cls

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ot3xbwl1 because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Camera Feed Widget

In [2]:
image_w = ipywidgets.Image(format='jpeg', width=640, height=480)\

# HTML Widget

In [3]:
#radar_widget = ipywidgets.Output()   # Uncomment this if you plan to visualize lidar data as a radar plot
radar_widget = ipywidgets.HTML()
distance_widget = ipywidgets.HTML()
deltaX_widget = ipywidgets.HTML()
deltaY_widget = ipywidgets.HTML()
angle_widget = ipywidgets.HTML()
stage_widget = ipywidgets.HTML()

In [4]:
# Visible widget (Text + Video Stream)
text_widget = ipywidgets.VBox(
    [distance_widget, deltaX_widget, deltaY_widget, angle_widget, stage_widget],
    layout=ipywidgets.Layout(width='50%')
)

full_widget = ipywidgets.HBox(
    [text_widget, radar_widget],
    layout=ipywidgets.Layout(width='100%')
)

image_widget = ipywidgets.HBox(
    [image_w],
    layout=ipywidgets.Layout(width='100%')
)

# Update State widgets

In [7]:
def updateStateWidgets():
    global distance_widget, deltaX_widget, deltaY_widget, angle_widget, stage_widget, rays_widget
    global observation, image_w, image
    
    angles = [x*10 for x in range(36)]                        
                        
    distance_widget.value = f"<pref><font color='red'><font size=6> Distance to person: {round(observation[0],2)}m"
    deltaX_widget.value = f"<pref><font color='black'><font size=6> Delta X: {round(observation[1],3)}m"
    deltaY_widget.value = f"<pref><font color='black'><font size=6> Delta Y: {round(observation[2],3)}m"
    angle_widget.value = f"<pref><font color='purple'><font size=6> Angle: {observation[3]}"
    stage_widget.value = f"<pref><font color='green'><font size=6> Stage: {observation[4]}"
    
    colors = ['black' if x/8 > 0.075 else 'red' for x in observation[5:]]
    
    radar_widget.value = f"""<pref><font color={colors[18]}><font size=6> 0 deg: {observation[23]} \
                                <br><font color={colors[22]}>40 deg: {observation[27]} \
                                <br><font color={colors[27]}>90 deg: {observation[32]} \
                                <br><font color={colors[31]}>130 deg: {observation[36]} \
                                <br><font color={colors[0]}>180 deg: {observation[5]} \
                                <br><font color={colors[4]}>220 deg: {observation[9]} \
                                <br><font color={colors[9]}>270 deg: {observation[14]} \
                                <br><font color={colors[13]}>310 deg: {observation[18]} """

    # Below is only used in case you want to visualize a lot of lidar readings. 
#     radar_widget.value = f"""<pref><font color='green'><font size=6> 0 deg: {previous_readings[0]} \
#                                 <br>10 deg: {previous_readings[10]} \
#                                 <br>20 deg: {previous_readings[20]} \
#                                 <br>30 deg: {previous_readings[30]} \
#                                 <br>40 deg: {previous_readings[40]} \
#                                 <br>50 deg: {previous_readings[50]} \
#                                 <br>60 deg: {previous_readings[60]} \
#                                 <br>70 deg: {previous_readings[70]} \
#                                 <br>80 deg: {previous_readings[80]} \
#                                 <br>90 deg: {previous_readings[90]} \
#                                 <br>100 deg: {previous_readings[100]} \
#                                 <br>110 deg: {previous_readings[110]} \
#                                 <br>120 deg: {previous_readings[120]} \
#                                 <br>130 deg: {previous_readings[130]} \
#                                 <br>140 deg: {previous_readings[140]} \
#                                 <br>150 deg: {previous_readings[150]} \
#                                 <br>160 deg: {previous_readings[160]} \
#                                 <br>170 deg: {previous_readings[170]} \
#                                 <br>180 deg: {previous_readings[180]} \
#                                 <br>190 deg: {previous_readings[190]} \
#                                 <br>200 deg: {previous_readings[200]} \
#                                 <br>210 deg: {previous_readings[210]} \
#                                 <br>220 deg: {previous_readings[220]} \
#                                 <br>230 deg: {previous_readings[230]} \
#                                 <br>240 deg: {previous_readings[240]} \
#                                 <br>250 deg: {previous_readings[250]} \
#                                 <br>260 deg: {previous_readings[260]} \
#                                 <br>270 deg: {previous_readings[270]} \
#                                 <br>280 deg: {previous_readings[280]} \
#                                 <br>290 deg: {previous_readings[290]} \
#                                 <br>300 deg: {previous_readings[300]} \
#                                 <br>310 deg: {previous_readings[310]} \
#                                 <br>320 deg: {previous_readings[320]} \
#                                 <br>330 deg: {previous_readings[330]} \
#                                 <br>340 deg: {previous_readings[340]} \
#                                 <br>350 deg: {previous_readings[350]} """
    
    # Uncomment if you want to visualize the lidar data as a radar
    #update_radar()
    
    # Send frame to video widget (only if you need to visualize what your camera is looking at.)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    
    

Below code is not used anymore. Before, I it used to display lidar readings as a radar. 
Sadly, the performance of the jetson xavier dropped considerably when using it.
So, I left it just for documentation purposes. 

In [6]:
def update_radar():
    global radar_widget, observation
       
    with radar_widget:
        
        fig, ax = radar.setup_axis()
        rings_inner = radar.draw_circles(ax=ax, facecolor='#ffffff', edgecolor='#000000')
        values = observation[5:]
        radar_poly, rings, vertices = radar.draw_radar(values, ax=ax,
                                                        kwargs_radar={'facecolor': '#00f2c1', 'alpha': 0.6}, 
                                                        kwargs_rings={'facecolor': '#00f2c1', 'alpha': 0.6})
        #range_labels = radar.draw_range_labels(ax=ax)
        param_labels = radar.draw_param_labels(ax=ax)

        radar_widget.clear_output(wait=True)
        plt.show()